In [17]:
import os
import json
import pandas as pd

In [18]:
path = '../results/02_bayesian_optimization'

# 1. Bayes Optimization Results

In [19]:
def results_parser(path):

    results_df = pd.DataFrame()

    for rd in os.listdir(path):

        curr_path = os.path.join(path, rd)

        # filtrando apenas os diretórios
        trials = [
            trial 
            for trial in os.listdir(curr_path) 
            if '.json' not in trial
        ]

        # abrindo os resultados de cada tentativa
        for trial in trials:
            curr_trial = os.path.join(curr_path, trial)

            file = os.path.join(curr_path, trial, [
                f
                for f in os.listdir(curr_trial)
                if '.json' in f
            ][0])

            with open(file, 'r') as f:
                curr_result = json.load(f)

            result = curr_result['hyperparameters']['values']
            result['epochs'] = curr_result['best_step']
            result['loss'] = curr_result['score']
            result['batch_size'] = int(rd.split('_')[-1])
            result['trial'] = int(trial.split('_')[1])

            results_df = pd.concat([
                results_df,
                pd.DataFrame(result, index=[0])
            ], ignore_index=True)
    
    return results_df.sort_values('loss')

## 1.1 Pretrain BERT

### 1.1.1 Book Reviews

In [20]:
# top 10 melhores resultados (bert com book reviews)
results_lit_review = results_parser(f'{path}/bert/book_reviews')
results_lit_review.head(10)

,hidden_size,num_hidden_layers,num_attention_heads,intermediate_size,hidden_dropout_prob,learning_rate,epochs,loss,batch_size,trial
51,768,6,4,1024,0.1,0.000067,5,6.372961,8,11
55,512,8,16,2048,0.2,0.000128,5,6.497189,8,15
56,512,10,16,2048,0.1,0.000075,5,6.511960,8,16
4,768,6,4,3072,0.1,0.000124,5,6.531558,16,4
40,768,8,12,2048,0.2,0.000048,5,6.532318,8,0
13,768,8,4,3072,0.1,0.000092,5,6.551585,16,13
57,256,12,4,1024,0.2,0.000120,5,6.638262,8,17
35,512,8,16,2048,0.3,0.000187,5,6.692791,32,15
6,512,6,16,3072,0.4,0.000149,5,6.718224,16,6
50,768,6,12,1024,0.5,0.000041,6,6.751915,8,10


In [21]:
# média de resultado por batch_size e arquitetura (bert com book reviews)
results_lit_review.groupby(
    ['batch_size']
).agg(
    {'loss': ['mean', 'std', 'median']}
).sort_values(['batch_size'])

loss                    
                mean       std    median
batch_size                              
8           6.862437  0.252776  6.936142
16          6.984916  0.204720  7.097733
32          7.015485  0.121421  7.053308

### 1.1.2 TV

In [22]:
# top 10 melhores resultados (bert com tv reviews)
results_tv_review = results_parser(f'{path}/bert/tv_reviews')
results_tv_review.head(10)

,hidden_size,num_hidden_layers,num_attention_heads,intermediate_size,hidden_dropout_prob,learning_rate,epochs,loss,batch_size,trial
21,768,6,12,1024,0.2,0.000102,9,3.492996,32,1
34,768,6,16,1024,0.4,0.000093,8,5.046766,32,14
54,512,10,4,2048,0.1,0.000040,7,5.071844,8,14
49,512,10,16,3072,0.1,0.000047,7,5.131644,8,9
9,768,10,12,2048,0.3,0.000038,7,5.411859,16,9
20,256,10,16,1024,0.2,0.000336,6,5.417662,32,0
11,512,12,16,1024,0.4,0.000094,6,5.459514,16,11
13,256,12,16,1024,0.4,0.000162,7,5.517826,16,13
40,768,6,4,1024,0.4,0.000041,5,5.533927,8,0
43,256,8,4,1024,0.2,0.000050,7,5.630235,8,3


In [23]:
# média de resultado por batch_size e arquitetura (bert com tv reviews)
results_tv_review.groupby(
    ['batch_size']
).agg(
    {'loss': ['mean', 'std', 'median']}
).sort_values(['batch_size'])

loss                    
                mean       std    median
batch_size                              
8           6.225119  0.648824  6.593739
16          6.325825  0.612352  6.773590
32          5.851689  0.902379  5.848976

## 1.2 Fine Tuning

In [24]:
def read_json(path: str) -> dict:
    """Reads a json file and returns a dictionary.
    
    Args:
        path: path to the json file

    Returns:
        dictionary with the contents of the json file
    """
    with open(path, "r") as f:
        return json.load(f)


def best_hyperparams(results: dict) -> dict:
    """Returns the best hyperparameters based on the F1 score.
    
    Args:
        results (dict): Dictionary with the results of the hyperparameter 
                        optimization.

    Returns:
        dict: Dictionary with the best hyperparameters.
    """
    best_values = {
        'trial': None,
        'F1': 0,
    }
    for trial, result in results.items():
        if result['value'] > best_values['F1']:
            best_values['trial'] = trial
            best_values['F1'] = result['value']

    print(f'Best trial: {best_values["trial"]}.')

    return results[best_values['trial']]

### 1.2.2 TV

**TODO: colocar a imagem da arquitetura aqui, para ficar mais claro.**

In [33]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results1 = read_json(f'{path}/fine_tuning/tv/baseline_aspect_extraction.json')
best_hyperparams(tv_results1)

Best trial: trial_5.


{'dropout_rate': 0.30000000000000004,
 'learning_rate': 4.356405490340622e-05,
 'value': 0.8242424242424243}

**TODO: colocar a imagem da arquitetura aqui, para ficar mais claro.**

In [26]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results2 = read_json(f'{path}/fine_tuning/tv/bert_visio.json')
best_hyperparams(tv_results2)

Best trial: trial_9.


{'dropout_rate': 0.5,
 'learning_rate': 0.00015039796790289732,
 'value': 0.7440944881889764}

**TODO: colocar a imagem da arquitetura aqui, para ficar mais claro.**

In [27]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results3 = read_json(f'{path}/fine_tuning/tv/double_bert_aspect_extraction.json')
best_hyperparams(tv_results3)

Best trial: trial_10.


{'dropout_rate': 0.30000000000000004,
 'learning_rate': 2.0863364780543777e-05,
 'value': 0.8501026694045174}

**TODO: colocar a imagem da arquitetura aqui, para ficar mais claro.**

In [28]:
# procurando qual tentativa apresentou o melhor desempenho
tv_results4 = read_json(f'{path}/fine_tuning/tv/double_bert_ffn_aspect_extraction.json')
best_hyperparams(tv_results4)

Best trial: trial_13.


{'dropout_rate': 0.4,
 'first_layer_units': 768,
 'second_layer_units': 1536,
 'third_layer_units': None,
 'learning_rate': 2.001383160164505e-05,
 'value': 0.830188679245283}

### 1.2.3 ReLi

In [35]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results1 = read_json(f'{path}/fine_tuning/reli/baseline_aspect_extraction.json')
best_hyperparams(reli_results1)

Best trial: trial_9.


{'dropout_rate': 0.1, 'learning_rate': 2e-05, 'value': 0.602}

In [30]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results2 = read_json(f'{path}/fine_tuning/reli/litera_bert.json')
best_hyperparams(reli_results2)

Best trial: trial_8.


{'dropout_rate': 0.5,
 'learning_rate': 3.282787355738218e-05,
 'value': 0.4633333333333333}

In [31]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results3 = read_json(f'{path}/fine_tuning/reli/double_bert_aspect_extraction.json')
best_hyperparams(reli_results3)

Best trial: trial_9.


{'dropout_raNonete': 0.5,
 'learning_rate': 2.6417102416521368e-05,
 'value': 0.6220735785953178}

In [32]:
# procurando qual tentativa apresentou o melhor desempenho
reli_results4 = read_json(f'{path}/fine_tuning/reli/double_bert_ffn_aspect_extraction.json')
best_hyperparams(reli_results4)

Best trial: trial_9.


{'dropout_rate': 0.0,
 'first_layer_units': 256,
 'second_layer_units': 512,
 'third_layer_units': 768,
 'learning_rate': 3.8534252343702257e-05,
 'value': 0.4692737430167598}